In [ ]:
import sys
import os

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import torch
import gpytorch
from tqdm.notebook import trange
import heapq
import math
import pickle
import itertools
from algorithms.cd import con_div
from algorithms.ccr import con_conv_rate
from utils.class_imbalance import get_classes, class_proportion

from algorithms.cgm import *

## Dataset

In [ ]:
(all_party_datasets, all_test_datasets, all_party_feats, all_test_feats, all_party_labels, all_test_labels, k) = pickle.load(open("MNIST-equaldisjoint-datasets_features_labels_kernel.p", "rb"))

In [ ]:
test_feats = np.concatenate(all_test_feats, axis=0)
test_labels = np.concatenate(all_test_labels, axis=0)

In [ ]:
num_parties = 5

In [ ]:
party_datasets = np.array(all_party_feats[1:])
party_labels = np.array(all_party_labels[1:])

In [ ]:
kernel = k

In [ ]:
perm_samp_dataset = np.concatenate(party_datasets)

In [ ]:
reference_dataset = np.concatenate(party_datasets)

In [ ]:
v = get_v(party_datasets, reference_dataset, kernel)

In [ ]:
v

In [ ]:
phi = shapley(v, num_parties)
print(phi)

In [ ]:
alpha = norm(phi)
print(alpha)

In [ ]:
vN = get_vN(v, num_parties)
print(vN)

In [ ]:
v_is = get_v_is(v, num_parties)
print(v_is)

## Max condition

In [ ]:
best_eta, q = get_eta_q(vN, alpha, v_is, perm_samp_dataset, reference_dataset, kernel, mode="max")

In [ ]:
best_eta

In [ ]:
v_is

In [ ]:
#all condition
r = list(map(q, alpha))
print(r)

In [ ]:
cand_datasets = np.tile(test_feats, (num_parties,1,1))

In [ ]:
greeds = np.ones(num_parties) * 3

In [ ]:
rewards, deltas, mus = reward_realization(cand_datasets, 
                                          reference_dataset, 
                                          r, 
                                          party_datasets, 
                                          kernel, 
                                          greeds=greeds)

In [ ]:
pickle.dump((reference_dataset, cand_datasets, party_datasets, greeds, rewards, deltas, mus), open("results/CGM-MNIST-equaldisjoint-greed3-max.p", "wb"))

In [ ]:
num_clusters=10

In [ ]:
class_props = []
for result in rewards:
    class_props.append(class_proportion(get_classes(np.array(result), test_feats, test_labels), num_clusters))

In [ ]:
class_props

In [ ]:
for i in range(num_parties):
    print(mmd_neg_biased(np.concatenate([party_datasets[i], np.array(rewards[i])], axis=0), reference_dataset, kernel)[0])

## All condition

In [ ]:
best_eta, q = get_eta_q(vN, alpha, v_is, perm_samp_dataset, reference_dataset, kernel, mode="all")

In [ ]:
best_eta

In [ ]:
v_is

In [ ]:
#all condition
r = list(map(q, alpha))
print(r)

In [ ]:
cand_datasets = np.tile(test_feats, (num_parties,1,1))

In [ ]:
greeds = np.ones(num_parties) * 2

In [ ]:
rewards, deltas, mus = reward_realization(cand_datasets, 
                                          reference_dataset, 
                                          r, 
                                          party_datasets, 
                                          kernel, 
                                          greeds=greeds)

In [ ]:
pickle.dump((gmm, clusters, reference_dataset, cand_datasets, party_datasets, greeds, rewards, deltas, mus), open("results/CGM-MNIST-equaldisjoint-greed2-all.p", "wb"))

In [ ]:
class_props = []
for result in rewards:
        class_props.append(class_proportion(get_classes(np.array(result), gmm[0], clusters[0]), num_clusters))

In [ ]:
class_props

In [ ]:
for i in range(num_parties):
    print(mmd_neg_biased(np.concatenate([party_datasets[i], np.array(rewards[i])], axis=0), reference_dataset, kernel)[0])